#### **Get Latest Videos and Comments**

import requests
import csv
from datetime import datetime, timedelta

class MovieReviewScraper:
    def __init__(self, tmdb_api_key, youtube_api_key):
        self.tmdb_api_key = tmdb_api_key
        self.youtube_api_key = youtube_api_key
        self.tmdb_base_url = "https://api.themoviedb.org/3"
        self.youtube_base_url = "https://www.googleapis.com/youtube/v3"
    
    def get_release_date(self, movie_name):
        """Get the release date of the movie from TMDB."""
        search_url = f"{self.tmdb_base_url}/search/movie"
        params = {
            "api_key": self.tmdb_api_key,
            "query": movie_name
        }
        response = requests.get(search_url, params=params)
        response.raise_for_status()
        data = response.json()

        if data['results']:
            release_date = data['results'][0]['release_date']
            country = data['results'][0].get('original_language', 'US')  # Default to US
            print(f"Movie Found: {movie_name} | Release Date: {release_date}")
            return release_date, country
        else:
            raise ValueError(f"Movie '{movie_name}' not found on TMDB.")

    def search_youtube_videos(self, movie_name, release_date, country_code):
        """Search for YouTube videos uploaded within a week of the movie release."""
        start_date = release_date
        end_date = (datetime.strptime(release_date, "%Y-%m-%d") + timedelta(days=7)).strftime("%Y-%m-%d")
        
        search_url = f"{self.youtube_base_url}/search"
        params = {
            "part": "snippet",
            "q": f"{movie_name} review",
            "type": "video",
            "regionCode": country_code,
            "publishedAfter": f"{start_date}T00:00:00Z",
            "publishedBefore": f"{end_date}T23:59:59Z",
            "key": self.youtube_api_key,
            "maxResults": 50  # Maximum results per page
        }
        response = requests.get(search_url, params=params)
        response.raise_for_status()
        data = response.json()

        video_ids = [item['id']['videoId'] for item in data.get('items', [])]
        print(f"Found {len(video_ids)} videos for '{movie_name}' reviews.")
        return video_ids

    def get_video_comments(self, video_id):
        """Fetch comments from a YouTube video."""
        comments = []
        comments_url = f"{self.youtube_base_url}/commentThreads"
        params = {
            "part": "snippet",
            "videoId": video_id,
            "key": self.youtube_api_key,
            "maxResults": 100  # Fetch maximum per page
        }

        while len(comments) < 200:  # Limit to 200 comments
            response = requests.get(comments_url, params=params)
            response.raise_for_status()
            data = response.json()

            for item in data.get('items', []):
                comments.append(item['snippet']['topLevelComment']['snippet']['textOriginal'])

            next_page_token = data.get('nextPageToken')
            if not next_page_token:
                break
            params['pageToken'] = next_page_token

        print(f"Fetched {len(comments)} comments for video ID: {video_id}")
        return comments[:200]

    def save_to_csv(self, movie_name, comments):
        """Save comments to a CSV file."""
        filename = f"{movie_name.replace(' ', '_')}_comments.csv"
        with open(filename, mode='w', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Comment"])
            writer.writerows([[comment] for comment in comments])
        print(f"Comments saved to {filename}")

    def scrape_movie_reviews(self, movie_name):
        """Main function to scrape movie reviews."""
        try:
            release_date, country = self.get_release_date(movie_name)
            video_ids = self.search_youtube_videos(movie_name, release_date, country)
            
            all_comments = []
            for video_id in video_ids:
                all_comments.extend(self.get_video_comments(video_id))
                if len(all_comments) >= 200:  # Limit total comments
                    break
            
            self.save_to_csv(movie_name, all_comments)
        except Exception as e:
            print(f"Error: {e}")

# Example usage:
# scraper = MovieReviewScraper("YOUR_TMDB_API_KEY", "YOUR_YOUTUBE_API_KEY")
# scraper.scrape_movie_reviews("Inception")
